<a href="https://colab.research.google.com/github/DeveloperPratim/CSV/blob/main/caffe_LightenNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Verify if gpu and nvidia drivers are present.

In [37]:
!nvidia-smi

Tue Jul 15 13:31:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Get caffe

In [38]:
# !sudo apt build-dep caffe-cuda
# This fails for me

In [39]:
!ls

models


In [40]:
!git clone https://github.com/BVLC/caffe.git
!git reset --hard 9b891540183ddc834a02b2bd81b31afae71b2153 #reset to the newest revision that worked OK on 27.03.2021

Cloning into 'caffe'...
remote: Enumerating objects: 65274, done.
remote: Total 65274 (delta 0), reused 0 (delta 0), pack-reused 65274 (from 1)
Receiving objects: 100% (65274/65274), 74.14 MiB | 11.77 MiB/s, done.
Resolving deltas: 100% (41241/41241), done.
HEAD is now at 9b891540 Merge pull request #6878 from timgates42/bugfix/typo_overridden


# Install Prepare dependencies

In [41]:
# !sudo apt-cache search libhdf5-
# !sudo apt-cache search gflags
# !sudo apt --fix-broken install
!sudo apt-get install libgflags2.2
!sudo apt-get install libgflags-dev
!sudo apt-get install libgoogle-glog-dev
# !sudo apt-get install libhdf5-10 - replaced with 100
!sudo apt-get install libhdf5-100
!sudo apt-get install libhdf5-serial-dev
!sudo apt-get install libhdf5-dev
# !sudo apt-get install libhdf5-cpp-11 - replaced with 100
!sudo apt-get install libhdf5-cpp-100
!sudo apt-get install libprotobuf-dev protobuf-compiler

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgflags2.2 is already the newest version (2.2.2-2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgflags-dev is already the newest version (2.2.2-2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgoogle-glog-dev is already the newest version (0.5.0+really0.4.0-2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libhdf5-100 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  libhdf5-hl-fo

In [42]:
!find /usr -iname "*hdf5.so"
# got: /usr/lib/x86_64-linux-gnu/hdf5/serial
!find /usr -iname "*hdf5_hl.so"

/usr/lib/x86_64-linux-gnu/libhdf5.so
/usr/lib/x86_64-linux-gnu/hdf5/serial/libhdf5.so
/usr/lib/x86_64-linux-gnu/libhdf5_hl.so
/usr/lib/x86_64-linux-gnu/hdf5/serial/libhdf5_hl.so


In [43]:
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial.so /usr/lib/x86_64-linux-gnu/libhdf5.so
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial_hl.so /usr/lib/x86_64-linux-gnu/libhdf5_hl.so

ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libhdf5.so': File exists
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libhdf5_hl.so': File exists


In [44]:
#!find /usr -iname "*hdf5.h*" # got:
# /usr/include/hdf5/serial/hdf5.h
# /usr/include/opencv2/flann/hdf5.h
# Let's try the first one.
%env CPATH="/usr/include/hdf5/serial/"
#fatal error: hdf5.h: No such file or directory

env: CPATH="/usr/include/hdf5/serial/"


In [45]:
!sudo apt-get install libleveldb-dev
!sudo apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
!sudo apt-get install libsnappy-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libleveldb-dev is already the newest version (1.23-3build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
liblmdb-dev is already the newest version (0.9.24-1build2).
libgflags-dev is already the newest version (2.2.2-2).
libgoogle-glog-dev is already the newest version (0.5.0+really0.4.0-2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsnappy-dev is already the newest version (1.1.8-1build3).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


# Build caffe from source files

In [46]:
!echo $CPATH

"/usr/include/hdf5/serial/"


In [47]:
%cd caffe
!ls
!make clean
!cp Makefile.config.example Makefile.config

/content/caffe/caffe/caffe
caffe.cloc	 data	   INSTALL.md		    models     tools
cmake		 docker    LICENSE		    python
CMakeLists.txt	 docs	   Makefile		    README.md
CONTRIBUTING.md  examples  Makefile.config.example  scripts
CONTRIBUTORS.md  include   matlab		    src
Makefile:6: *** Makefile.config not found. See Makefile.config.example..  Stop.


## Use sed for editing Makefile.config
Just so we don't have to manually open the file with text editor and look where to add the lines below.


In [49]:
!sed -i 's/-gencode arch=compute_20/#-gencode arch=compute_20/' Makefile.config #old cuda versions won't compile
!sed -i 's/\/usr\/local\/include/\/usr\/local\/include \/usr\/include\/hdf5\/serial\//'  Makefile.config #one of the 4 things needed to fix hdf5 issues
!sed -i 's/# OPENCV_VERSION := 3/OPENCV_VERSION := 3/' Makefile.config #We actually use opencv 4.1.2, but it's similar enough to opencv 3.
!sed -i 's/code=compute_61/code=compute_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_75,code=compute_75/' Makefile.config #support for new GPUs

In [50]:
!apt-get update
!apt-get install -y libopencv-dev

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [51]:
!make all -j 4 # -j would use all availiable cores, but RAM related errors occur

PROTOC src/caffe/proto/caffe.proto
NVCC src/caffe/layers/tanh_layer.cu
NVCC src/caffe/layers/concat_layer.cu
NVCC src/caffe/layers/sigmoid_layer.cu
NVCC src/caffe/layers/eltwise_layer.cu
NVCC src/caffe/layers/scale_layer.cu
NVCC src/caffe/layers/cudnn_pooling_layer.cu
NVCC src/caffe/layers/softmax_layer.cu
NVCC src/caffe/layers/im2col_layer.cu
NVCC src/caffe/layers/swish_layer.cu
NVCC src/caffe/layers/lrn_layer.cu
NVCC src/caffe/layers/batch_norm_layer.cu
NVCC src/caffe/layers/accuracy_layer.cu
NVCC src/caffe/layers/softmax_loss_layer.cu
NVCC src/caffe/layers/elu_layer.cu
NVCC src/caffe/layers/power_layer.cu
NVCC src/caffe/layers/lstm_unit_layer.cu
NVCC src/caffe/layers/inner_product_layer.cu
NVCC src/caffe/layers/crop_layer.cu
NVCC src/caffe/layers/cudnn_conv_layer.cu
NVCC src/caffe/layers/contrastive_loss_layer.cu
NVCC src/caffe/layers/batch_reindex_layer.cu
NVCC src/caffe/layers/cudnn_tanh_layer.cu
NVCC src/caffe/layers/bnll_layer.cu
NVCC src/caffe/layers/log_layer.cu
NVCC src/caffe

In [52]:
!rm examples/cpp_classification/classification.cpp

In [56]:
!rm examples/mnist/convert_mnist_data.cpp
!rm src/caffe/test/test_memory_data_layer.cpp

In [58]:
!rm -rf examples/cpp_classification examples/mnist examples/siamese \
       tools/compute_image_mean.cpp tools/upgrade_*.cpp \
       tools/convert_imageset.cpp tools/extract_features.cpp

In [53]:
%cd /content/caffe
!make clean
!make all -j4
!make test -j4

/content/caffe
PROTOC src/caffe/proto/caffe.proto
NVCC src/caffe/layers/tanh_layer.cu
NVCC src/caffe/layers/concat_layer.cu
NVCC src/caffe/layers/sigmoid_layer.cu
NVCC src/caffe/layers/eltwise_layer.cu
NVCC src/caffe/layers/scale_layer.cu
NVCC src/caffe/layers/cudnn_pooling_layer.cu
NVCC src/caffe/layers/softmax_layer.cu
NVCC src/caffe/layers/im2col_layer.cu
NVCC src/caffe/layers/swish_layer.cu
NVCC src/caffe/layers/lrn_layer.cu
NVCC src/caffe/layers/batch_norm_layer.cu
NVCC src/caffe/layers/accuracy_layer.cu
NVCC src/caffe/layers/softmax_loss_layer.cu
NVCC src/caffe/layers/elu_layer.cu
NVCC src/caffe/layers/power_layer.cu
NVCC src/caffe/layers/lstm_unit_layer.cu
NVCC src/caffe/layers/inner_product_layer.cu
NVCC src/caffe/layers/crop_layer.cu
NVCC src/caffe/layers/cudnn_conv_layer.cu
NVCC src/caffe/layers/contrastive_loss_layer.cu
NVCC src/caffe/layers/batch_reindex_layer.cu
NVCC src/caffe/layers/cudnn_tanh_layer.cu
NVCC src/caffe/layers/bnll_layer.cu
NVCC src/caffe/layers/log_layer.cu

In [61]:

from IPython.display import display, HTML

# Read the Makefile.config content
with open('/content/caffe/Makefile.config', 'r') as f:
    content = f.read()

# Display with copy button
display(HTML(f"""
<textarea id="makefile_config" style="width:100%; height:400px;">{content}</textarea>
<br>
<button onclick="navigator.clipboard.writeText(document.getElementById('makefile_config').value)">📋 Copy Makefile.config</button>
"""))

In [67]:

makefile_config = """
## Google Colab-friendly Makefile.config for GPU (cuDNN), Python3, OpenCV 4

# Enable cuDNN
USE_CUDNN := 1

# OpenCV version
OPENCV_VERSION := 3  # Keep this, but we fix include path below

# CUDA path
CUDA_DIR := /usr/local/cuda

# CUDA architectures (for Volta/Turing/Ampere GPUs in Colab)
CUDA_ARCH := -gencode arch=compute_70,code=sm_70 \\
             -gencode arch=compute_75,code=sm_75 \\
             -gencode arch=compute_80,code=sm_80 \\
             -gencode arch=compute_86,code=sm_86

# BLAS (ATLAS is simplest)
BLAS := atlas

# Python 3 headers and NumPy
PYTHON_INCLUDE := /usr/include/python3.10 \\
                  /usr/lib/python3/dist-packages/numpy/core/include

# Python lib
PYTHON_LIB := /usr/lib

# Enable Python layers
WITH_PYTHON_LAYER := 1

# Include and library paths
INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial/ /usr/include/opencv4
LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib

# Output paths
BUILD_DIR := build
DISTRIBUTE_DIR := distribute

# GPU to use for test
TEST_GPUID := 0

# Quiet build
Q ?= @
"""

In [68]:

with open("/content/caffe/Makefile.config", "w") as f:
    f.write(makefile_config.strip())

print("[✓] Updated Makefile.config with OpenCV 4 support for Colab.")

[✓] Updated Makefile.config with OpenCV 4 support for Colab.


In [64]:
!sed -i '/^SUBDIRS/s/examples//g' /content/caffe/Makefile
!sed -i '/^SUBDIRS/s/test//g' /content/caffe/Makefile
!echo "[✓] Patched Makefile to skip examples/ and test/"

[✓] Patched Makefile to skip examples/ and test/


In [74]:
!find /usr/include -name core.hpp

/usr/include/boost/callable_traits/detail/core.hpp
/usr/include/boost/phoenix/core.hpp
/usr/include/boost/compute/core.hpp
/usr/include/boost/compute/interop/eigen/core.hpp
/usr/include/boost/compute/interop/opencv/core.hpp
/usr/include/boost/move/core.hpp
/usr/include/boost/proto/core.hpp
/usr/include/boost/spirit/home/x3/core.hpp
/usr/include/boost/spirit/home/classic/core.hpp
/usr/include/boost/beast/core.hpp
/usr/include/boost/log/core/core.hpp
/usr/include/boost/log/core.hpp
/usr/include/boost/hana/fwd/core.hpp
/usr/include/boost/hana/core.hpp
/usr/include/boost/lambda/core.hpp
/usr/include/opencv4/opencv2/core/core.hpp
/usr/include/opencv4/opencv2/core.hpp


In [69]:

%cd /content/caffe
!make clean
!make all -j4
!make pycaffe -j4

/content/caffe
PROTOC src/caffe/proto/caffe.proto
NVCC src/caffe/layers/tanh_layer.cu
NVCC src/caffe/layers/concat_layer.cu
NVCC src/caffe/layers/sigmoid_layer.cu
NVCC src/caffe/layers/eltwise_layer.cu
NVCC src/caffe/layers/scale_layer.cu
NVCC src/caffe/layers/cudnn_pooling_layer.cu
NVCC src/caffe/layers/softmax_layer.cu
NVCC src/caffe/layers/im2col_layer.cu
NVCC src/caffe/layers/swish_layer.cu
NVCC src/caffe/layers/lrn_layer.cu
NVCC src/caffe/layers/batch_norm_layer.cu
NVCC src/caffe/layers/accuracy_layer.cu
NVCC src/caffe/layers/softmax_loss_layer.cu
NVCC src/caffe/layers/elu_layer.cu
NVCC src/caffe/layers/power_layer.cu
NVCC src/caffe/layers/lstm_unit_layer.cu
NVCC src/caffe/layers/inner_product_layer.cu
NVCC src/caffe/layers/crop_layer.cu
NVCC src/caffe/layers/cudnn_conv_layer.cu
NVCC src/caffe/layers/contrastive_loss_layer.cu
NVCC src/caffe/layers/batch_reindex_layer.cu
NVCC src/caffe/layers/cudnn_tanh_layer.cu
NVCC src/caffe/layers/bnll_layer.cu
NVCC src/caffe/layers/log_layer.cu

# Verify if caffe works

In [70]:
import cv2
print(cv2.__version__)
import caffe

4.12.0


## LightenNet

In [71]:

import os

def write_train_val_prototxt(path):
    text = '''
name: "LightenNet"

layer {
  name: "data"
  type: "HDF5Data"
  top: "data"
  top: "label"
  include { phase: TRAIN }
  hdf5_data_param {
    source: "data/hdf5/train.txt"
    batch_size: 16
  }
}

layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 64
    kernel_size: 9
    stride: 1
    pad: 4
    weight_filler { type: "xavier" }
    bias_filler { type: "constant" }
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "conv1"
  top: "conv1"
}

layer {
  name: "conv2"
  type: "Convolution"
  bottom: "conv1"
  top: "conv2"
  convolution_param {
    num_output: 32
    kernel_size: 5
    stride: 1
    pad: 2
    weight_filler { type: "xavier" }
    bias_filler { type: "constant" }
  }
}
layer {
  name: "relu2"
  type: "ReLU"
  bottom: "conv2"
  top: "conv2"
}

layer {
  name: "conv3"
  type: "Convolution"
  bottom: "conv2"
  top: "conv3"
  convolution_param {
    num_output: 16
    kernel_size: 5
    stride: 1
    pad: 2
    weight_filler { type: "xavier" }
    bias_filler { type: "constant" }
  }
}
layer {
  name: "relu3"
  type: "ReLU"
  bottom: "conv3"
  top: "conv3"
}

layer {
  name: "conv4"
  type: "Convolution"
  bottom: "conv3"
  top: "output"
  convolution_param {
    num_output: 1
    kernel_size: 5
    stride: 1
    pad: 2
    weight_filler { type: "xavier" }
    bias_filler { type: "constant" }
  }
}

layer {
  name: "loss"
  type: "EuclideanLoss"
  bottom: "output"
  bottom: "label"
  top: "loss"
}
'''
    with open(path, 'w') as f:
        f.write(text)
    print(f"[✓] train_val.prototxt written to {path}")


def write_solver_prototxt(path):
    text = '''
net: "caffe/models/lightennet/train_val.prototxt"
base_lr: 0.001
momentum: 0.9
weight_decay: 0.0005
lr_policy: "step"
stepsize: 5000
gamma: 0.1
display: 100
max_iter: 20000
snapshot: 5000
snapshot_prefix: "caffe/models/lightennet/lightennet"
solver_mode: GPU
'''
    with open(path, 'w') as f:
        f.write(text)
    print(f"[✓] solver.prototxt written to {path}")


# Create folder and write files
os.makedirs("caffe/models/lightennet", exist_ok=True)
write_train_val_prototxt("caffe/models/lightennet/train_val.prototxt")
write_solver_prototxt("caffe/models/lightennet/solver.prototxt")

[✓] train_val.prototxt written to caffe/models/lightennet/train_val.prototxt
[✓] solver.prototxt written to caffe/models/lightennet/solver.prototxt


In [ ]:

!unzip -q /content/caffe/low_light_data.zip -d /content/caffe/low_light_data
!find /content/caffe/low_light_data -type d

/content/caffe/low_light_data
/content/caffe/low_light_data/our485
/content/caffe/low_light_data/our485/high
/content/caffe/low_light_data/our485/low
/content/caffe/low_light_data/__MACOSX
/content/caffe/low_light_data/__MACOSX/our485
/content/caffe/low_light_data/__MACOSX/our485/high
/content/caffe/low_light_data/__MACOSX/our485/low
/content/caffe/low_light_data/__MACOSX/eval15
/content/caffe/low_light_data/__MACOSX/eval15/high
/content/caffe/low_light_data/__MACOSX/eval15/low
/content/caffe/low_light_data/eval15
/content/caffe/low_light_data/eval15/high
/content/caffe/low_light_data/eval15/low


In [ ]:

import os
import cv2
import numpy as np
import h5py

LOW_DIR = "/content/caffe/low_light_data/our485/low"
HIGH_DIR = "/content/caffe/low_light_data/our485/high"
SAVE_DIR = "/content/caffe/data/hdf5/train"

os.makedirs(SAVE_DIR, exist_ok=True)

valid_extensions = (".png", ".jpg", ".jpeg", ".bmp")

file_list = []
for i, fname in enumerate(sorted(os.listdir(LOW_DIR))):
    if not fname.lower().endswith(valid_extensions):
        print(f"Skipping non-image file {fname}")
        continue

    low_path = os.path.join(LOW_DIR, fname)
    high_path = os.path.join(HIGH_DIR, fname)

    if not os.path.isfile(high_path):
        print(f"High-light image missing for {fname}, skipping.")
        continue

    low_img = cv2.imread(low_path, cv2.IMREAD_COLOR)
    high_img = cv2.imread(high_path, cv2.IMREAD_COLOR)

    if low_img is None or high_img is None:
        print(f"Failed to read image pair for {fname}, skipping.")
        continue

    low_img = cv2.resize(low_img, (128, 128))
    high_img = cv2.resize(high_img, (128, 128))

    low_img = low_img.astype(np.float32) / 255.0
    high_img = high_img.astype(np.float32) / 255.0

    # Change from (H, W, C) to (C, H, W)
    low_img = low_img.transpose(2, 0, 1)
    high_img = high_img.transpose(2, 0, 1)

    # Add batch dimension
    low_img = low_img[np.newaxis, ...]
    high_img = high_img[np.newaxis, ...]

    h5_filename = os.path.join(SAVE_DIR, f"pair_{i}.h5")
    with h5py.File(h5_filename, 'w') as h5f:
        h5f.create_dataset('data', data=low_img)
        h5f.create_dataset('label', data=high_img)

    file_list.append(h5_filename)

with open(os.path.join(SAVE_DIR, 'train.txt'), 'w') as f:
    for item in file_list:
        f.write(item + '\n')

print(f"[✓] Created {len(file_list)} HDF5 training samples in {SAVE_DIR}")

Skipping non-image file .DS_Store
[✓] Created 485 HDF5 training samples in /content/caffe/data/hdf5/train


In [ ]:

import os
import cv2

LOW_DIR = "/content/caffe/low_light_data/our485/low"
for fname in sorted(os.listdir(LOW_DIR)):
    if fname.lower().endswith((".png", ".jpg", ".bmp")):
        path = os.path.join(LOW_DIR, fname)
        img = cv2.imread(path)
        if img is None:
            print(f"Failed to read: {fname}")
        else:
            print(f"{fname}: shape={img.shape}")

10.png: shape=(400, 600, 3)
100.png: shape=(400, 600, 3)
101.png: shape=(400, 600, 3)
102.png: shape=(400, 600, 3)
103.png: shape=(400, 600, 3)
104.png: shape=(400, 600, 3)
105.png: shape=(400, 600, 3)
106.png: shape=(400, 600, 3)
107.png: shape=(400, 600, 3)
109.png: shape=(400, 600, 3)
110.png: shape=(400, 600, 3)
112.png: shape=(400, 600, 3)
113.png: shape=(400, 600, 3)
114.png: shape=(400, 600, 3)
115.png: shape=(400, 600, 3)
116.png: shape=(400, 600, 3)
117.png: shape=(400, 600, 3)
118.png: shape=(400, 600, 3)
119.png: shape=(400, 600, 3)
12.png: shape=(400, 600, 3)
120.png: shape=(400, 600, 3)
121.png: shape=(400, 600, 3)
122.png: shape=(400, 600, 3)
123.png: shape=(400, 600, 3)
124.png: shape=(400, 600, 3)
125.png: shape=(400, 600, 3)
126.png: shape=(400, 600, 3)
127.png: shape=(400, 600, 3)
128.png: shape=(400, 600, 3)
129.png: shape=(400, 600, 3)
13.png: shape=(400, 600, 3)
130.png: shape=(400, 600, 3)
131.png: shape=(400, 600, 3)
132.png: shape=(400, 600, 3)
135.png: shape=(4

In [ ]:

import os
import cv2
import numpy as np
import h5py

LOW_DIR = "/content/caffe/low_light_data/eval15/low"
HIGH_DIR = "/content/caffe/low_light_data/eval15/high"
SAVE_DIR = "/content/caffe/data/hdf5/val"

os.makedirs(SAVE_DIR, exist_ok=True)

valid_extensions = (".png", ".jpg", ".jpeg", ".bmp")

file_list = []
for i, fname in enumerate(sorted(os.listdir(LOW_DIR))):
    if not fname.lower().endswith(valid_extensions):
        print(f"Skipping non-image file {fname}")
        continue

    low_path = os.path.join(LOW_DIR, fname)
    high_path = os.path.join(HIGH_DIR, fname)

    if not os.path.isfile(high_path):
        print(f"High-light image missing for {fname}, skipping.")
        continue

    low_img = cv2.imread(low_path, cv2.IMREAD_COLOR)
    high_img = cv2.imread(high_path, cv2.IMREAD_COLOR)

    if low_img is None or high_img is None:
        print(f"Failed to read image pair for {fname}, skipping.")
        continue

    low_img = cv2.resize(low_img, (128, 128))
    high_img = cv2.resize(high_img, (128, 128))

    low_img = low_img.astype(np.float32) / 255.0
    high_img = high_img.astype(np.float32) / 255.0

    low_img = low_img.transpose(2, 0, 1)
    high_img = high_img.transpose(2, 0, 1)

    low_img = low_img[np.newaxis, ...]
    high_img = high_img[np.newaxis, ...]

    h5_filename = os.path.join(SAVE_DIR, f"pair_{i}.h5")
    with h5py.File(h5_filename, 'w') as h5f:
        h5f.create_dataset('data', data=low_img)
        h5f.create_dataset('label', data=high_img)

    file_list.append(h5_filename)

with open(os.path.join(SAVE_DIR, 'val.txt'), 'w') as f:
    for item in file_list:
        f.write(item + '\n')

print(f"[✓] Created {len(file_list)} validation HDF5 samples in {SAVE_DIR}")
print()

[✓] Created 15 validation HDF5 samples in /content/caffe/data/hdf5/val



In [81]:

%cd /content/caffe

/content/caffe


In [87]:
import sys
sys.path.insert(0, "/content/caffe/python")

import caffe

In [97]:

import os

tool_dir = "/content/caffe/build/tools"
if os.path.exists(tool_dir):
    files = os.listdir(tool_dir)
    if files:
        print(f"[✓] Files in {tool_dir}:")
        for f in files:
            print("•", f)
    else:
        print(f"[!] No files found in {tool_dir}")
else:
    print(f"[✗] Directory not found: {tool_dir}")

[✓] Files in /content/caffe/build/tools:
• caffe.o
• caffe.d
• caffe.o.warnings.txt
